<a href="https://colab.research.google.com/github/SnSabu/congressional-analysis/blob/main/Flight_Data_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prepping input data

In [ ]:
import pandas as pd
import datetime
import time
import random
import re


In [ ]:
Office_Airports = pd.read_csv("Closest_airport_to_hubs.csv")
Emp_Airport = pd.read_csv("closest_airports_employees.csv")

In [ ]:
hub_airport = pd.concat([Office_Airports['Airport 1'], Office_Airports['Airport 2']])
hub_airport = pd.DataFrame(hub_airport, columns=['Destination Airport'])

emp_airport = pd.concat([pd.Series(Emp_Airport['Airport 1'].unique()), pd.Series(Emp_Airport['Airport 2'].unique())])
emp_airport = pd.DataFrame(emp_airport, columns=['Origin Airport'])


hub_airport['key'] = 1
emp_airport['key'] = 1
airport_combinations = pd.merge(emp_airport, hub_airport, on ='key').drop("key", 1)


#Data cleaning
airport_combinations = airport_combinations.dropna().reset_index(drop=True)
airport_combinations= airport_combinations.drop_duplicates().reset_index(drop=True)
#Removing rows with same origin and destination airport location
mask = airport_combinations['Origin Airport'] != airport_combinations['Destination Airport']
# select only the rows where the mask is True
airport_combinations = airport_combinations.loc[mask]

<ipython-input-150-0221a84e686e>:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  airport_combinations = pd.merge(emp_airport, hub_airport, on ='key').drop("key", 1)


In [ ]:
airport_combinations['departure_date'] = '2023-03-17'

Using Duffel

In [ ]:
!pip install duffel-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from duffel_api import Duffel

#duffel_test_mSO5C-rVg_mIuSj_co7IND-CIcuZ_ak0I8M1X-jas3A
access_token = 'duffel_test_2d04g-56TplXGi5pPmsg2idnrQxG0VkT-caelLdpFcX'
client = Duffel(access_token = access_token)

In [ ]:
import re
import datetime
import time
import random

# Define a function to concatenate the values in each row
def airport_prices(row):
    destination = row['Destination Airport']
    origin = row['Origin Airport']
    departure_date = row['departure_date']

    slices = [
    {
        "origin": origin,
        "destination": destination,
        "departure_date": departure_date,
    },
    ]
    offer_request = (
        client.offer_requests.create()
        .passengers([{"type": "adult"}])
        .slices(slices)
        .return_offers()
        .execute()
    )
    offers = offer_request.offers
    #Getting the top airline details
        
    if offers[0].owner.name == "Duffel Airways" and len(offers) > 1 and offers[1] is not None:
      i = 1
    else:
      i = 0
    
#    while i < len(offers):
#        dept_time = offers[i].slices[0].segments[0].departing_at.time()
#        if 6 <= dept_time.hour < 21:
#            #print(f"Offer {i} has a departure time between 6AM and 9PM")            
#            break    
#        i += 1

#    if i >= len(offers):
#      i=1

    offer = offers[i] 

    #Extracting the flight duration in minutes
    duration_string = offer.slices[0].segments[0].duration
        
    # Regular expression to match hours and minutes in duration strings
    pattern = re.compile(r"PT(?:(\d+)H)?(?:(\d+)M)?")
    match = pattern.match(duration_string)
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    total_minutes =  hours * 60 + minutes
    #duration = datetime.timedelta(seconds=datetime.datetime.strptime(duration_string, "PT%HH%MM").timestamp())
    # Convert the timedelta to minutes and output as a numeric type
    #total_minutes = int(duration.total_seconds() // 60)
    
    #Extracting flight info
    flight_price = float(offer.total_amount)
    airline = offer.owner.name
    departure_time = offers[i].slices[0].segments[0].departing_at.time()
    arrival_time = offers[i].slices[0].segments[0].arriving_at.time()
    
    return total_minutes, flight_price, airline, departure_time, arrival_time

#for index, row in airport_combinations.iloc[1059:].iterrows():
for index, row in airport_combinations.iterrows():
    print("This is the air travel details for the row number: {} ".format(index))
    airport_combinations.loc[index, ['Duration', 'Price', 'Airlines', 'Departure Time', 'Arrival Time']] = airport_prices(row)
    print(row)
    

# Print the resulting dataframe
print(airport_combinations)

# Save the DataFrame to a CSV file
airport_combinations.to_csv('Flight_Prices_2023_03_17_Any_hr.csv', index=False)


This is the air travel details for the row number: 0 
Origin Airport                BOS
Destination Airport           ATL
departure_date         2023-03-17
Duration                      NaN
Price                         NaN
Airlines                      NaN
Departure Time                NaN
Arrival Time                  NaN
Name: 0, dtype: object
This is the air travel details for the row number: 1 
Origin Airport                BOS
Destination Airport           SEA
departure_date         2023-03-17
Duration                      NaN
Price                         NaN
Airlines                      NaN
Departure Time                NaN
Arrival Time                  NaN
Name: 1, dtype: object
This is the air travel details for the row number: 2 
Origin Airport                BOS
Destination Airport           ORD
departure_date         2023-03-17
Duration                      NaN
Price                         NaN
Airlines                      NaN
Departure Time                NaN
Arrival Ti

ApiError: ignored

Rough Work

In [ ]:
# Save the DataFrame to a CSV file
airport_combinations.to_csv('Flight_Prices_2023_03_17.csv', index=False)

In [ ]:

origin = 'FAI'
destination = 'SFO'
departure_date = '2023-03-18'
print("\nSearching flights...")
slices = [
    {
        "origin": origin,
        "destination": destination,
        "departure_date": departure_date,
    },
]
offer_request = (
    client.offer_requests.create()
    .passengers([{"type": "adult"}])
    .slices(slices)
    .return_offers()
    .execute()
)
offers = offer_request.offers
for idx, offer in enumerate(offers):
    print(
        f"{idx + 1}. {offer.owner.name} flight departing at "
        + f"{offer.slices[0].segments[0].departing_at} duration of "
         + f"{offer.slices[0].segments[0].duration} "
        + f"{offer.total_amount} {offer.total_currency}"
        )
        
    




Searching flights...
1. Duffel Airways flight departing at 2023-03-18 13:09:00 duration of PT5H19M 183.05 USD
2. Delta Air Lines flight departing at 2023-03-18 01:30:00 duration of PT3H37M 271.20 USD
3. Delta Air Lines flight departing at 2023-03-18 01:30:00 duration of PT3H37M 330.60 USD
4. Delta Air Lines flight departing at 2023-03-18 01:30:00 duration of PT3H37M 440.87 USD
5. Alaska Airlines flight departing at 2023-03-18 02:45:00 duration of PT3H34M 514.80 USD
6. Alaska Airlines flight departing at 2023-03-18 15:50:00 duration of PT3H34M 589.80 USD
7. Alaska Airlines flight departing at 2023-03-18 02:45:00 duration of PT3H34M 614.80 USD
8. Alaska Airlines flight departing at 2023-03-18 07:10:00 duration of PT3H35M 659.80 USD
9. Alaska Airlines flight departing at 2023-03-18 15:50:00 duration of PT3H34M 689.80 USD
10. Alaska Airlines flight departing at 2023-03-18 07:10:00 duration of PT3H35M 759.80 USD
11. Delta Air Lines flight departing at 2023-03-18 01:30:00 duration of PT3H37

In [ ]:


destination = 'PVD'
origin = 'SEA'
departure_date = '2023-03-17'
print("\nSearching flights...")
slices = [
    {
        "origin": origin,
        "destination": destination,
        "departure_date": departure_date,
    },
]
offer_request = (
    client.offer_requests.create()
    .passengers([{"type": "adult"}])
    .slices(slices)
    .return_offers()
    .execute()
)

offers = offer_request.offers
#Getting the top airline details

if len(offers) > 1 and offers[1] is not None:
  offer = offers[1]
else:
  offer = offers[0]


#Extracting the flight duration in minutes
duration_string = offer.slices[0].segments[0].duration

# Extract the hours and minutes from the duration string using regex
matches = re.findall(r'PT(\d+)H(\d+)M', duration_string)

if matches:
    # Get the hours and minutes from the matches
    hours, minutes = int(matches[0][0]), int(matches[0][1])
    # Create a timedelta object with the hours and minutes
    duration = datetime.timedelta(hours=hours, minutes=minutes)
    # Calculate the total minutes
    total_minutes = duration.total_seconds() // 60

else:
    matches = re.findall(r'PT(\d*)H(\d+)M', duration_string)
    # Get the hours and minutes from the matches
    minutes = int(matches[0][1])
    # Create a timedelta object with the hours and minutes
    duration = datetime.timedelta(minutes=minutes)
    # Calculate the total minutes
    total_minutes = duration.total_seconds() // 60
    
print(total_minutes)

#Extracting flight price
flight_price = float(offer.total_amount)
print(flight_price)

airline = offer.owner.name
print(airline)
    


Searching flights...
265.0
248.2
Delta Air Lines
